In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt 

2024-07-24 15:01:41.052330: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 15:01:41.054951: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 15:01:41.105409: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 15:01:41.106863: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 15:01:41.881004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
PATH = "../kaggle/data/"
# PATH = "/kaggle/input/isic-2024-challenge/"


In [3]:
df_0 = pd.read_csv(PATH + "train-metadata.csv")

/tmp/ipykernel_52207/1877868789.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_0 = pd.read_csv(PATH + "train-metadata.csv")


In [4]:
# Clean and preprocess data for training and testing 

cols_to_drop = ["isic_id", "patient_id", "mel_thick_mm", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "attribution", "copyright_license", "tbp_lv_location", "tbp_lv_location_simple", "tbp_lv_dnn_lesion_confidence", "anatom_site_general", "image_type", "tbp_tile_type"]
df_0.drop(columns=cols_to_drop, inplace=True)
df_0["sex"] = df_0["sex"].replace({"male": 1, "female": 0})

# replace all NaNs with np.nan
df_0.replace("", np.nan, inplace=True)

# Drop any row with a NaN value
df_0.dropna(inplace=True)

# Drop any column with a string vallue 
for col in df_0.columns:
    if df_0[col].dtype == "object":
        df_0.drop(columns=[col], inplace=True)

# clean up for training, split into train and test
train_frac = 0.8 
train_size = int(train_frac * len(df_0))

train = df_0[:train_size]
x_train = df_0.drop(columns=["target"])
y_train = df_0["target"]

test = df_0[train_size:]
x_test = df_0.drop(columns=["target"])
y_test = df_0["target"]

In [5]:
# Define the NN 
inputs = keras.Input(shape=(len(x_train.keys()),)) # One input for each feature in the dataset 

# Add two hidden layers with 64 units each and ReLU activation
x = layers.Dense(64, activation="relu")(inputs)
x = layers.Dense(64, activation="relu")(x)

# Define the output layer with a single unit and sigmoid activation
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="isic-v0")


2024-07-24 15:01:47.494125: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 15:01:47.533373: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[keras.metrics.BinaryAccuracy()],
)

history = model.fit(x_train, y_train, batch_size=512, epochs=10, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/10


/home/ubuntu/miniconda3/envs/tf-v2/lib/python3.8/site-packages/keras/src/backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 [==============================] - 1s 707ms/step - loss: 0.3268 - binary_accuracy: 0.9756 - val_loss: 2.4304e-06 - val_binary_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 24ms/step - loss: 1.0978e-11 - binary_accuracy: 1.0000 - val_loss: 2.4304e-06 - val_binary_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 23ms/step - loss: 1.0978e-11 - binary_accuracy: 1.0000 - val_loss: 2.4304e-06 - val_binary_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 23ms/step - loss: 1.0978e-11 - binary_accuracy: 1.0000 - val_loss: 2.4304e-06 - val_binary_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 31ms/step - loss: 1.0978e-11 - binary_accuracy: 1.0000 - val_loss: 2.4304e-06 - val_binary_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - ETA: 0s - loss: 1.0978e-11 - binary_accuracy: 1.0000

/home/ubuntu/miniconda3/envs/tf-v2/lib/python3.8/site-packages/keras/src/backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 [==============================] - 0s 47ms/step - loss: 1.0978e-11 - binary_accuracy: 1.0000 - val_loss: 2.4304e-06 - val_binary_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 32ms/step - loss: 1.0978e-11 - binary_accuracy: 1.0000 - val_loss: 2.4304e-06 - val_binary_accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 32ms/step - loss: 1.0978e-11 - binary_accuracy: 1.0000 - val_loss: 2.4304e-06 - val_binary_accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 31ms/step - loss: 1.0978e-11 - binary_accuracy: 1.0000 - val_loss: 2.4304e-06 - val_binary_accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 30ms/step - loss: 1.0978e-11 - binary_accuracy: 1.0000 - val_loss: 2.4304e-06 - val_binary_accuracy: 1.0000
2/2 - 0s - loss: 5.1413e-07 - binary_accuracy: 1.0000 - 28ms/epoch - 14ms/step
Test loss: 5.141284304954752e-07
Test accuracy: 1.0


In [7]:
test_df = pd.read_csv(PATH + "test-metadata.csv")
isic_ids = test_df["isic_id"]

try: 
    test_df.drop(columns=cols_to_drop, inplace=True)
except KeyError: 
    pass

test_df.replace("", np.nan, inplace=True)
test_df["sex"] = test_df["sex"].replace({"male": 1, "female": 0})
test_df.dropna(inplace=True)

# Drop any column with a string vallue 
for col in test_df.columns:
    if test_df[col].dtype == "object":
        test_df.drop(columns=[col], inplace=True)

In [13]:
print(len(test_df.keys()), len(x_train.keys()))

35 35


In [9]:
preds = model.predict(test_df)

df_sub = pd.DataFrame({"isic_id": isic_ids, "target": preds[:,0]})
df_sub.to_csv("submission.csv", index=False)


1/1 [==============================] - 0s 64ms/step


In [10]:
df_sub

,isic_id,target
0,ISIC_0015657,1.0
1,ISIC_0015729,1.0
2,ISIC_0015740,1.0
